In [ ]:
import xarray as xr
import cartopy
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from datetime import datetime
import os.path
import matplotlib.pyplot as plt
import cartopy.feature as cfeature
from PIL import Image
import glob
url = 'http://map1c.vis.earthdata.nasa.gov/wmts-geo/wmts.cgi'
layer = 'BlueMarble_NextGeneration'

#workdir = ""/Users/james/Documents/Github/esm_lab/MOM6/analysis/output/nwa25/Cheyenne/2021-11-05/"
workdir = "/glade/work/jsimkins/runs-MOM6/nwa25/"

ds = xr.open_mfdataset(workdir + "20100*")

var_list = ['sst', 'ssh050', 'sss']

for var in var_list:
    print(var)
    for i in range(0,len(ds.time.values)):
        if os.path.isfile(workdir + "/images/" + var + datetime.strptime(str(ds.time[i].values), "%Y-%m-%d %H:%M:%S").strftime("%Y%m%d%H%M") + ".png") == False:
            figure = plt.figure(figsize=(12,12))
            ax = figure.add_subplot(1,1,1, projection=crs.PlateCarree())
            ax.add_wmts(url, layer)
            ax.set_extent([-100, -34, 3,54], crs=crs.PlateCarree())
            if var=='sst':
                im = ax.pcolormesh(ds.xh.values, ds.yh.values, ds.sst[i].values, transform=crs.PlateCarree(), cmap='turbo', vmin=0, vmax=30)
                ax.add_wmts(url, 'BlueMarble_NextGeneration')
                plt.colorbar(im, label="Sea Surface Temperature (deg C)", shrink=0.6, pad=.03)
                ax.gridlines()
                ax.text(0.2, 1.015, "NWA25 Sea Surface Temperature - " + str(ds.time[i].values), size=12, transform=ax.transAxes)
                plt.savefig(workdir + "/images/" + var + datetime.strptime(str(ds.time[i].values), "%Y-%m-%d %H:%M:%S").strftime("%Y%m%d%H%M") + ".png")
                plt.close()
            if var=='ssh050':
                im = ax.pcolormesh(ds.xh.values, ds.yh.values, ds.ssh[i].values, transform=crs.PlateCarree(), cmap='turbo', vmin=-0.50, vmax=0.50)
                ax.add_wmts(url, 'BlueMarble_NextGeneration')
                plt.colorbar(im, label="Sea Surface Height (m)", shrink=0.6, pad=.03)
                ax.gridlines()
                ax.text(0.2, 1.015, "NWA25 Sea Surface Height - " + str(ds.time[i].values), size=12, transform=ax.transAxes)
                plt.savefig(workdir + "/images/" + var + datetime.strptime(str(ds.time[i].values), "%Y-%m-%d %H:%M:%S").strftime("%Y%m%d%H%M") + ".png")
                plt.close()
            if var=='sss':
                im = ax.pcolormesh(ds.xh.values, ds.yh.values, ds.sss[i].values, transform=crs.PlateCarree(), cmap='turbo', vmin=30, vmax=40)
                ax.add_wmts(url, 'BlueMarble_NextGeneration')
                plt.colorbar(im, label="Sea Surface Salinity (psu)", shrink=0.6, pad=.03)
                ax.gridlines()
                ax.text(0.2, 1.015, "NWA25 Sea Surface Salinity - " + str(ds.time[i].values), size=12, transform=ax.transAxes)
                plt.savefig(workdir + "/images/" + var + datetime.strptime(str(ds.time[i].values), "%Y-%m-%d %H:%M:%S").strftime("%Y%m%d%H%M") + ".png")
                plt.close()

    img_names = sorted(glob.glob(workdir + "/images/" + var + "*.png"))
    dur_vals = []
    for i in range(0,len(img_names)):
        if i != len(img_names)-1:
            dur_vals.append(200)
    dur_vals.append(3000)

    img, *imgs = [Image.open(f) for f in img_names]
    img.save(fp=workdir + var + '.gif', format='GIF', append_images=imgs,
         save_all=True, duration=dur_vals, loop=0)
